In [1]:
import os
import time
import pandas as pd
import urllib.request
from itertools import chain
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.common.exceptions import NoSuchElementException





In [37]:

os.environ['PATH'] = f'{os.environ["PATH"]}:{os.getcwd()}/drivers'
ean_list= ['8710103880547', '8436028922536', '8436551456690', '3397701245487', '8435178361905']


In [38]:
def scroll_shim(passed_in_driver, object):
    x = object.location['x']
    y = object.location['y']
    scroll_by_coord = 'window.scrollTo(%s,%s);' % ( x, y)
    scroll_nav_out_of_way = 'window.scrollBy(0, -73);'
    passed_in_driver.execute_script(scroll_by_coord)
    passed_in_driver.execute_script(scroll_nav_out_of_way)
    
def feature_function(features):
    features_list=features.split("\n")
    long=len(features_list)
    count_till= 40  
    while long < count_till:
        features_list.append(float('nan'))
        long+=1
    return features_list


def initialize_eci_info_df():
    
    eci_info = pd.DataFrame(columns = ['item_name.value', 
                                                          'list_price.value_with_tax', 
                                                          'main_image','features','more_features'
                                                         ])
    return eci_info


In [51]:
def eci(ean_list):
    
    eci_info = initialize_eci_info_df()
    
    browser = webdriver.Firefox()
    browser.get('https://www.elcorteingles.es/')

    time.sleep(3)

    
    for ean in ean_list:
        
        
        title = price = main_image = features = more_features = float('nan')
        
    
        search = browser.find_element_by_xpath('//input[@id="search-box"]')
        search.clear()
        search.send_keys(ean)
        search.send_keys(Keys.ENTER)

        time.sleep(3)
        
        
        try:

            title = browser.find_elements_by_xpath('//h2[@class="title"]')[0].text
            
            
        except NoSuchElementException:
            
            print(f"title {ean} not found")
            
        except IndexError:
            
            print(f"title {ean} not found")
            
        
        
        try: 
            
            price = browser.find_elements_by_xpath('//div[@class="product-price "]')[0].text
            price_list=price.split("€")
            price = price_list[0]
        
        except NoSuchElementException:
            
            print(f"price {ean} not found")
        
        except IndexError:
            
            print(f"price {ean} not found")
            
        
        try: 
            
             main_image = browser.find_element_by_xpath('//img[@id="product-image-placer"]').get_attribute('src')

        except NoSuchElementException:
            
            print(f"main image {ean} not found")
        
        except IndexError:
            
            print(f"main image {ean} not found")
            
        
        
        try: 
            
            customer_reviews = browser.find_elements_by_xpath('//button[@id="star-button"]')
            for my_aria_label in customer_reviews:
                customer_reviews =my_aria_label.get_attribute("aria-label")
        
        except NoSuchElementException:
            
            print(f" customer reviews {ean} not found")
        
        except IndexError:
            
            print(f" customer reviews  image {ean} not found")
            

        try:

            action = ActionChains(browser)
            click_view_more  = browser.find_element_by_xpath("(//span[@id='text-view-more'])")
            
            if 'firefox' in browser.capabilities['browserName']:
                scroll_shim(browser, click_view_more)
            action.move_to_element((click_view_more)).click().perform()

         
        except NoSuchElementException:
            
            print("IMPOSILE TO SCROLL")
                        
            
        except IndexError:
            
            print("IMPOSILE TO SCROLL")
        
        
        
        try:   
            
            features = browser.find_elements_by_xpath('//div[@id="features"]')[0].text
        
        
        except NoSuchElementException:
            
            print(f" features {ean} not found")
                        
            
        except IndexError:
            
            print(f" features {ean} not found")
            
            
        
        try:
            
            more_features = browser.find_elements_by_xpath('//div[@id="inpage_container"]')[0].text

        
        except NoSuchElementException: 

            print(f" more features {ean} not found")
            
            
        except IndexError:
            
            print(f" more features {ean} not found")
            
            
        
        eci_info.loc[ean] = [title, price, main_image, features, more_features]
    
    
    browser.close()
    
    return eci_info
                



In [49]:
df = eci(ean_list)

title 8436028922536 not found
main image 8436028922536 not found
IMPOSILE TO SCROLL
 features 8436028922536 not found
 more features 8436028922536 not found
title 8436551456690 not found
main image 8436551456690 not found
IMPOSILE TO SCROLL
 features 8436551456690 not found
 more features 8436551456690 not found
title 3397701245487 not found
main image 3397701245487 not found
IMPOSILE TO SCROLL
 features 3397701245487 not found
 more features 3397701245487 not found
title 8435178361905 not found
main image 8435178361905 not found
IMPOSILE TO SCROLL
 features 8435178361905 not found
 more features 8435178361905 not found


In [50]:
df

,item_name.value,list_price.value_with_tax,main_image,features,more_features
8710103880547,Plancha de vapor Philips GC4564/20 con suela S...,59,https://sgfm.elcorteingles.es/SGFM/dctm/MEDIA0...,Plancha de vapor con sistema Quick Calc Releas...,Rendimiento del vapor garantizado\nCon nuestro...
8436028922536,NaN,139,NaN,Plancha de vapor con sistema Quick Calc Releas...,Rendimiento del vapor garantizado\nCon nuestro...
8436551456690,NaN,139,NaN,Plancha de vapor con sistema Quick Calc Releas...,Rendimiento del vapor garantizado\nCon nuestro...
3397701245487,NaN,139,NaN,Plancha de vapor con sistema Quick Calc Releas...,Rendimiento del vapor garantizado\nCon nuestro...
8435178361905,NaN,139,NaN,Plancha de vapor con sistema Quick Calc Releas...,Rendimiento del vapor garantizado\nCon nuestro...
